## SIADS591 - Milestone I - Project
### Exploring the Factors Impacting a Movie's Profitability
#### Machine Learning  - Regression

In [17]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR
import matplotlib.pyplot as plt
%matplotlib inline

The prepared data is imported:

In [18]:
save_dir = "../../Output/"
imdb_dummies_df = pd.read_csv(save_dir + "imdb_dummies_df_2020_09_15.csv", sep="|")
print("imdb_dummies_df.shape :", imdb_dummies_df.shape)
cols = list(imdb_dummies_df.columns)
print("cols :", cols[:10])

imdb_dummies_df.shape : (4797, 29711)
cols : ['tconst', 'primaryTitle', 'originalTitle', 'startYear', 'imdb_id', 'revenue', 'budget', 'runtime', 'director__A.R. Murugadoss', 'director__Aanand L. Rai']


The target variable is created (y).

In [19]:
ml_df = imdb_dummies_df.copy()
y = ml_df["revenue"] / ml_df["budget"]

The features are prepared in a variable called X.

In [20]:
ml_df["startYear"] = ml_df["startYear"].astype(int).astype(str)
ml_df.drop(["primaryTitle", "originalTitle", "imdb_id", "revenue", "budget"], axis="columns", inplace=True)
ml_df.set_index("tconst", inplace=True)
print(ml_df[list(ml_df.columns)[:5]].head())

X = ml_df

features = list(X.columns)
print("len(features) :", len(features))
if "revenue" in features:
    print("revenue in features!")
# print("y :", y.head())

          startYear  runtime  director__A.R. Murugadoss  \
tconst                                                    
tt0035423      2001    118.0                          0   
tt0081455      1981    103.0                          0   
tt0081633      1981    116.0                          0   
tt0081974      1981    116.0                          0   
tt0082005      1981    112.0                          0   

           director__Aanand L. Rai  director__Aaron Blaise  
tconst                                                      
tt0035423                        0                       0  
tt0081455                        0                       0  
tt0081633                        0                       0  
tt0081974                        0                       0  
tt0082005                        0                       0  
len(features) : 29705


In [21]:
print("y.shape :", y.shape)
print("X.shape :", X.shape)

y.shape : (4797,)
X.shape : (4797, 29705)


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [23]:
est = SVR()

print("est.get_params().keys() ", est.get_params().keys())
grid_dict = {"kernel": ["linear"],  # "linear", ‘poly’, ‘rbf’, ‘sigmoid’"
             "degree": [1],
             "C": [0.1],
             "gamma": ["auto"]
             }
# grid_dict = {}
grid = GridSearchCV(estimator=est,
                    param_grid=grid_dict,
                    scoring="r2",
                    n_jobs=2,
                    cv=5)

est.get_params().keys()  dict_keys(['C', 'cache_size', 'coef0', 'degree', 'epsilon', 'gamma', 'kernel', 'max_iter', 'shrinking', 'tol', 'verbose'])


In [24]:
grid.fit(X_train, y_train)
best_params = grid.best_params_
print("best_params :", best_params)
# feature_importances = grid.best_estimator_.feature_importances_
# print("feature_importances :", feature_importances)
# feature_importances_df = pd.DataFrame({"Feature_name": features, "Score": feature_importances})
# feature_importances_df.sort_values("Score", ascending=False, inplace=True)
# print("feature_importances_df :")
# print(feature_importances_df.head(10))

y_pred = grid.predict(X_test)
score = grid.score(X_test, y_test)
print("score :", score)
mae = mean_absolute_error(y_true=y_test, y_pred=y_pred)
print("mae :", mae)

best_params : {'C': 0.1, 'degree': 1, 'gamma': 'auto', 'kernel': 'linear'}


AttributeError: 'SVR' object has no attribute 'feature_importances_'

In [ ]:
plt.clf()
plt.scatter(y_pred, y_test, marker="o", color="blue")
plt.xlabel("Predicted Values")
plt.ylabel("Actual Values")
# plt.savefig(save_dir + "Predicted_vs_Actual.png")
plt.show()